# Downloading the dataset

We are using the Face Forensics++ (FF++) dataset

In [2]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("sanikatiwarekar/deep-fake-detection-dfd-entire-original-dataset")

# print("Path to dataset files:", path)

/Users/jordan/Desktop/Live_Projects/History_of_Man/proactives_venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 22.5G/22.5G [23:54<00:00, 16.8MB/s]  


Extracting files...
Path to dataset files: /Users/jordan/.cache/kagglehub/datasets/sanikatiwarekar/deep-fake-detection-dfd-entire-original-dataset/versions/1


No CSV file found in dataset directory: /Users/jordan/.cache/kagglehub/datasets/sanikatiwarekar/deep-fake-detection-dfd-entire-original-dataset/versions/1


In [ ]:
import os
from dotenv import load_dotenv
#store the paths to the real and fake images in an env file

load_dotenv()

path = os.getenv('path')
fake_path = os.getenv('DEEPFAKE_PATH')

def count_mp4_files(folder):
    mp4_count = 0
    for root, dirs, files in os.walk(folder):
        mp4_count += sum(1 for f in files if f.endswith('.mp4'))
    return mp4_count

real_mp4_count = count_mp4_files(path)
fake_mp4_count = count_mp4_files(fake_path)

print(f'Real folder mp4 count: {real_mp4_count}')
print(f'Fake folder mp4 count: {fake_mp4_count}')

Real folder mp4 count: 363
Fake folder mp4 count: 3068


Use the EfficientNet model for feature extraction

In [8]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
import numpy as np

def get_efficientnet_feature_extractor():
    """
    Returns an EfficientNetB0 model that outputs feature vectors instead of class predictions.
    """
    # Load EfficientNetB0 without the top classification layer
    base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
    # The output will be the global average pooled features
    feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)
    return feature_extractor

def extract_face_features(face_images, feature_extractor):
    """
    Given a batch of face images, returns their EfficientNet feature vectors.
    face_images: numpy array of shape (batch_size, height, width, channels)
    feature_extractor: model returned by get_efficientnet_feature_extractor()
    """
    # EfficientNet expects images scaled to [0, 255] and size 224x224
    # You may need to preprocess your images accordingly
    features = feature_extractor.predict(face_images)
    return features

In [47]:
import cv2
import os
import numpy as np
import glob
from ultralytics import YOLO

def save_video_sequence_as_npy(video_path, output_dir, yolo_model_path='yolov8n.pt', conf=0.5, save_images=False):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return
    os.makedirs(output_dir, exist_ok=True)
    if save_images:
        os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    
    model = YOLO(yolo_model_path)
    frame_count = 0
    video_sequence = []  # Store all frames in a list
    
    while frame_count < 210:
        ret, frame = cap.read()
        if not ret:
            print(f"Could not read frame {frame_count}")
            break
        
        results = model.predict(source=frame, conf=conf, classes=0, verbose=False)
        
        # Find the largest face in this frame
        largest_face = None
        largest_area = 0
        
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                area = (x2 - x1) * (y2 - y1)
                
                if area > largest_area:
                    largest_area = area
                    largest_face = (x1, y1, x2, y2)
        
        # Process and add frame to sequence if face found
        if largest_face:
            x1, y1, x2, y2 = largest_face
            face = frame[y1:y2, x1:x2]
            face_resized = cv2.resize(face, (224, 224))
            face_scaled = face_resized.astype(np.float32) / 127.5 - 1.0
            
            # Add to video sequence
            video_sequence.append(face_scaled)
            
            # Optionally save individual images for visualization
            if save_images:
                face_image = ((face_scaled + 1.0) * 127.5).astype(np.uint8)
                cv2.imwrite(os.path.join(output_dir, 'images', f"face_{frame_count:04d}.png"), face_image)
        else:
            print(f"No face detected in frame {frame_count}")
            # You could optionally add a zero/placeholder frame here
            # placeholder = np.zeros((224, 224, 3), dtype=np.float32)
            # video_sequence.append(placeholder)
        
        frame_count += 1
    
    cap.release()
    
    # Convert list to numpy array and save as single .npy file
    if video_sequence:
        video_array = np.array(video_sequence)  # Shape: (num_frames, 224, 224, 3)
        
        # Get video name for the .npy file
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        npy_path = os.path.join(output_dir, f"{video_name}.npy")
        
        np.save(npy_path, video_array)
        print(f"Saved video sequence: {video_array.shape} to {npy_path}")
    else:
        print("No faces detected in any frame - no .npy file saved")


def process_videos(path, name):
    if not path or not os.path.exists(path):
        print("Path not found or doesn't exist")
        return
    
    # Find all mp4 files in the folder
    video_files = glob.glob(os.path.join(path, "*.mp4"))
    
    if not video_files:
        print("No mp4 files found in folder")
        return
    
    print(f"Found {len(video_files)} videos to process")
    
    # Create main output directory
    output_base = f'processed_{name}_sequences'
    os.makedirs(output_base, exist_ok=True)
    
    for i, video_path in enumerate(video_files):
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        print(f"Processing video {i+1}/{len(video_files)}: {video_name}")
        
        try:
            save_video_sequence_as_npy(video_path, output_base, save_images=False)
        except Exception as e:
            print(f"Error processing {video_name}: {e}")
            continue
    
    print(f"Finished processing all videos in {path}")

In [48]:
# import os

# # Get the path to the fake videos folder from your .env
# fake_path = os.getenv('DEEPFAKE_PATH')

# # List all mp4 files in the fake folder
# fake_videos = [f for f in os.listdir(fake_path) if f.endswith('.mp4')]

# if fake_videos:
#     first_video = fake_videos[0]
#     video_path = os.path.join(fake_path, first_video)
#     output_dir = 'example_output'  # Change this to your desired output folder

#     print(f"Processing video: {video_path}")
#     save_first_210_cropped_faces(video_path, output_dir, save_images=True)
# else:
#     print("No fake videos found in the folder.")

In [ ]:
# import glob

# def process_videos(path, name):
    
#     if not path or not os.path.exists(path):
#         print("Path not found or doesn't exist")
#         return
    
#     # Find all mp4 files in the real folder
#     video_files = glob.glob(os.path.join(path, "*.mp4"))
    
#     if not video_files:
#         print("No mp4 files found in folder")
#         return
    
#     print(f"Found {len(video_files)} videos to process")
    
#     for i, video_path in enumerate(video_files):
#         # Get video filename without extension
#         video_name = os.path.splitext(os.path.basename(video_path))[0]
        
#         # Create output directory for this video
#         output_dir = os.path.join(f'processed_{name}_videos', video_name)
        
#         print(f"Processing video {i+1}/{len(video_files)}: {video_name}")
        
#         try:
#             save_first_210_cropped_faces(video_path, output_dir, save_images=False)
#         except Exception as e:
#             print(f"Error processing {video_name}: {e}")
#             continue
    
#     print(f"Finished processing all videos in {path}")

In [49]:
import cv2
import os
import numpy as np
from ultralytics import YOLO

def process_single_video_to_mp4(video_path, output_path, yolo_model_path='yolov8n.pt', conf=0.5):
    """
    Process a single video, crop faces, and save as MP4
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = 224, 224  # Output resolution
    
    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    model = YOLO(yolo_model_path)
    frame_count = 0
    
    print(f"Processing video: {video_path}")
    print(f"Output: {output_path}")
    
    while frame_count < 210:
        ret, frame = cap.read()
        if not ret:
            print(f"Could not read frame {frame_count}")
            break
        
        results = model.predict(source=frame, conf=conf, classes=0, verbose=False)
        
        # Find the largest face in this frame
        largest_face = None
        largest_area = 0
        
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                area = (x2 - x1) * (y2 - y1)
                
                if area > largest_area:
                    largest_area = area
                    largest_face = (x1, y1, x2, y2)
        
        # Process and write frame
        if largest_face:
            x1, y1, x2, y2 = largest_face
            face = frame[y1:y2, x1:x2]
            face_resized = cv2.resize(face, (224, 224))
            
            # Write the cropped face to output video
            out.write(face_resized)
            print(f"Frame {frame_count}: Face detected and saved")
        else:
            # Write a black frame if no face detected
            black_frame = np.zeros((224, 224, 3), dtype=np.uint8)
            out.write(black_frame)
            print(f"Frame {frame_count}: No face detected, black frame saved")
        
        frame_count += 1
    
    cap.release()
    out.release()
    print(f"Finished processing. Output saved to: {output_path}")

# Example usage
fake_path = os.getenv('DEEPFAKE_PATH')
fake_videos = [f for f in os.listdir(fake_path) if f.endswith('.mp4')]

if fake_videos:
    first_video = fake_videos[0]
    video_path = os.path.join(fake_path, first_video)
    
    # Create output filename
    video_name = os.path.splitext(first_video)[0]
    output_path = f"processed_{video_name}_faces.mp4"
    
    process_single_video_to_mp4(video_path, output_path)
else:
    print("No fake videos found in the folder.")

Processing video: /Users/jordan/.cache/kagglehub/datasets/sanikatiwarekar/deep-fake-detection-dfd-entire-original-dataset/versions/1/DFD_manipulated_sequences/DFD_manipulated_sequences/13_20__walking_down_indoor_hall_disgust__EV1V4ZQV.mp4
Output: processed_13_20__walking_down_indoor_hall_disgust__EV1V4ZQV_faces.mp4
Frame 0: Face detected and saved
Frame 1: Face detected and saved
Frame 2: Face detected and saved
Frame 3: Face detected and saved
Frame 4: Face detected and saved
Frame 5: Face detected and saved
Frame 6: Face detected and saved
Frame 7: Face detected and saved
Frame 8: Face detected and saved
Frame 9: Face detected and saved
Frame 10: Face detected and saved
Frame 11: Face detected and saved
Frame 12: Face detected and saved
Frame 13: Face detected and saved
Frame 14: Face detected and saved
Frame 15: Face detected and saved
Frame 16: Face detected and saved
Frame 17: Face detected and saved
Frame 18: Face detected and saved
Frame 19: Face detected and saved
Frame 20: Fac

KeyboardInterrupt: 

In [ ]:
real = os.getenv('REAL_PATH')
fake = os.getenv('DEEPFAKE_PATH')

process_videos(real, "real")
process_videos(fake, "fake")

Found 363 videos to process
Processing video 1/363: 07__exit_phone_room
Saved video sequence: (210, 224, 224, 3) to processed_real_sequences/07__exit_phone_room.npy
Processing video 2/363: 09__kitchen_pan
Saved video sequence: (210, 224, 224, 3) to processed_real_sequences/09__kitchen_pan.npy
Processing video 3/363: 02__walking_down_street_outside_angry
Saved video sequence: (210, 224, 224, 3) to processed_real_sequences/02__walking_down_street_outside_angry.npy
Processing video 4/363: 12__talking_angry_couch
Saved video sequence: (210, 224, 224, 3) to processed_real_sequences/12__talking_angry_couch.npy
Processing video 5/363: 11__podium_speech_happy
Saved video sequence: (210, 224, 224, 3) to processed_real_sequences/11__podium_speech_happy.npy
Processing video 6/363: 26__podium_speech_happy
Saved video sequence: (210, 224, 224, 3) to processed_real_sequences/26__podium_speech_happy.npy
Processing video 7/363: 24__kitchen_still
Saved video sequence: (210, 224, 224, 3) to processed_re